In [1]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + 'dataImport/')
from utils import workingDataPath, getWorkingHRDfParquet, getHRGroups, calcIntersectionOfMultipleGroups, getHRsForTimePeriods, graphMultiHRDate, graphHypnoandHRDate
import pandas as pd
from datetime import date, time, timedelta

In [2]:
appleHRDf = getWorkingHRDfParquet('apple')
print(len(appleHRDf))
polarHRDf = getWorkingHRDfParquet('polar')
print(len(polarHRDf))
fitbitHRDf = getWorkingHRDfParquet('fitbit')
print(len(fitbitHRDf))

567895
1903183
16052785


In [3]:
# takes 15 sec to run
#let's look at times that have samples 15 seconds or less apart
appleGroups = getHRGroups(appleHRDf, 15)
print(len(appleGroups))
polarGroups = getHRGroups(polarHRDf, 15)
print(len(polarGroups))
fitbitGroups = getHRGroups(fitbitHRDf, 15)
print(len(fitbitGroups))

650
82
28616


In [4]:
apple_fitbit_intersections = calcIntersectionOfMultipleGroups([appleGroups, fitbitGroups])
print(len(apple_fitbit_intersections))
apple_polar_intersections = calcIntersectionOfMultipleGroups([appleGroups, polarGroups])
print(len(apple_polar_intersections))
fitbit_polar_intersections = calcIntersectionOfMultipleGroups([polarGroups, fitbitGroups])
print(len(fitbit_polar_intersections))
all_intersections = calcIntersectionOfMultipleGroups([appleGroups, fitbitGroups, polarGroups])
print(len(all_intersections))

653
80
244
132


In [20]:
def squaredDiffForTwoGroupsNoInterpolation(intersectingGroups, HRDf1, HRDf2):
    intersectingGroupsDf =  pd.DataFrame(intersectingGroups, columns = ['startDate', 'endDate'])
    HRs1 = getHRsForTimePeriods(intersectingGroupsDf, HRDf1)
    HRs2 = getHRsForTimePeriods(intersectingGroupsDf, HRDf2)
    print(f"number of samples in each group {len(HRs1)} {len(HRs2)}")

    #for now lets just calc intersections with the same index using the join function
    intersectingHRs = HRs1.join(HRs2, how='inner', lsuffix="_1", rsuffix="_2")
    print(f"number of samples in common {len(intersectingHRs)}")
    squaredDiff = ((intersectingHRs['value_1'] - intersectingHRs['value_2']) ** 2).sum()
    print(f"total Squared Difference: {squaredDiff}")
    return squaredDiff/len(intersectingHRs)


In [21]:
print(squaredDiffForTwoGroupsNoInterpolation(apple_fitbit_intersections, appleHRDf, fitbitHRDf))
print(squaredDiffForTwoGroupsNoInterpolation(apple_polar_intersections, appleHRDf, polarHRDf))
print(squaredDiffForTwoGroupsNoInterpolation(fitbit_polar_intersections, polarHRDf, fitbitHRDf))

bumber of samples in each group 234030 173203
number of samples in common 33861
total Squared Difference: 5114113.0
151.03254481556954
bumber of samples in each group 99389 496848
number of samples in common 99355
total Squared Difference: 4511979.0
45.41270192743194
bumber of samples in each group 1464704 200121
number of samples in common 200047
total Squared Difference: 38348586
191.69788099796548


In [24]:
def squaredDiffForTwoGroupsInterpolation(intersectingGroups, HRDf1, HRDf2, interpolationType):
    intersectingGroupsDf =  pd.DataFrame(intersectingGroups, columns = ['startDate', 'endDate'])
    HRs1 = getHRsForTimePeriods(intersectingGroupsDf, HRDf1)
    HRs2 = getHRsForTimePeriods(intersectingGroupsDf, HRDf2)

    HRs1Interp = HRs1.resample('1s').asfreq().interpolate(method=interpolationType)
    HRs2Interp = HRs2.resample('1s').asfreq().interpolate(method=interpolationType)
    print(f"number of samples in each group {len(HRs1Interp)} {len(HRs2Interp)}")

    #for now lets just calc intersections with the same index using the join function
    intersectingHRs = HRs1Interp.join(HRs2Interp, how='inner', lsuffix="_1", rsuffix="_2")
    print(f"number of samples in common {len(intersectingHRs)}")
    squaredDiff = ((intersectingHRs['value_1'] - intersectingHRs['value_2']) ** 2).sum()
    print(f"total Squared Difference: {squaredDiff}")
    return squaredDiff/len(intersectingHRs)

In [25]:
print(squaredDiffForTwoGroupsInterpolation(apple_fitbit_intersections, appleHRDf, fitbitHRDf, 'linear'))
print(squaredDiffForTwoGroupsInterpolation(apple_polar_intersections, appleHRDf, polarHRDf, 'linear'))
print(squaredDiffForTwoGroupsInterpolation(fitbit_polar_intersections, polarHRDf, fitbitHRDf, 'linear'))

number of samples in each group 99791385 99791387
number of samples in common 99791382
total Squared Difference: 20060817492.214104
201.02755458596718
number of samples in each group 16552143 16552146
number of samples in common 16552143
total Squared Difference: 29094777873.89336
1757.7650141068357
number of samples in each group 136078706 136078705
number of samples in common 136078705
total Squared Difference: 12100593495.23862
88.92349096971948


In [26]:
print(squaredDiffForTwoGroupsInterpolation(apple_fitbit_intersections, appleHRDf, fitbitHRDf, 'nearest'))
print(squaredDiffForTwoGroupsInterpolation(apple_polar_intersections, appleHRDf, polarHRDf, 'nearest'))
print(squaredDiffForTwoGroupsInterpolation(fitbit_polar_intersections, polarHRDf, fitbitHRDf, 'nearest'))

number of samples in each group 99791385 99791387
number of samples in common 99791382
total Squared Difference: 28949200733.0
290.09720231151823
number of samples in each group 16552143 16552146
number of samples in common 16552143
total Squared Difference: 41167550203.0
2487.1432178298605
number of samples in each group 136078706 136078705
number of samples in common 136078705
total Squared Difference: 20663212488.0
151.84750977752176


In [27]:
print(squaredDiffForTwoGroupsInterpolation(apple_fitbit_intersections, appleHRDf, fitbitHRDf, 'zero'))
print(squaredDiffForTwoGroupsInterpolation(apple_polar_intersections, appleHRDf, polarHRDf, 'zero'))
print(squaredDiffForTwoGroupsInterpolation(fitbit_polar_intersections, polarHRDf, fitbitHRDf, 'zero'))

number of samples in each group 99791385 99791387
number of samples in common 99791382
total Squared Difference: 9844806817.0
98.65387791703296
number of samples in each group 16552143 16552146
number of samples in common 16552143
total Squared Difference: 18206414856.0
1099.9430621158842
number of samples in each group 136078706 136078705
number of samples in common 136078705
total Squared Difference: 34350305819.0
252.4296936761707


In [29]:
print(squaredDiffForTwoGroupsInterpolation(apple_fitbit_intersections, appleHRDf, fitbitHRDf, 'pad'))
print(squaredDiffForTwoGroupsInterpolation(apple_polar_intersections, appleHRDf, polarHRDf, 'pad'))
print(squaredDiffForTwoGroupsInterpolation(fitbit_polar_intersections, polarHRDf, fitbitHRDf, 'pad'))

C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:6: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs1Interp = HRs1.resample('1s').asfreq().interpolate(method=interpolationType)
C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:7: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs2Interp = HRs2.resample('1s').asfreq().interpolate(method=interpolationType)


number of samples in each group 99791385 99791387
number of samples in common 99791382
total Squared Difference: 9844806817.0
98.65387791703296


C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:6: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs1Interp = HRs1.resample('1s').asfreq().interpolate(method=interpolationType)
C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:7: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs2Interp = HRs2.resample('1s').asfreq().interpolate(method=interpolationType)


number of samples in each group 16552143 16552146
number of samples in common 16552143
total Squared Difference: 18206414856.0
1099.9430621158842


C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:6: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs1Interp = HRs1.resample('1s').asfreq().interpolate(method=interpolationType)
C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:7: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs2Interp = HRs2.resample('1s').asfreq().interpolate(method=interpolationType)


number of samples in each group 136078706 136078705
number of samples in common 136078705
total Squared Difference: 34350305819.0
252.4296936761707


In [30]:
print(squaredDiffForTwoGroupsInterpolation(apple_fitbit_intersections, appleHRDf, fitbitHRDf, 'bfill'))
print(squaredDiffForTwoGroupsInterpolation(apple_polar_intersections, appleHRDf, polarHRDf, 'bfill'))
print(squaredDiffForTwoGroupsInterpolation(fitbit_polar_intersections, polarHRDf, fitbitHRDf, 'bfill'))

C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:6: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs1Interp = HRs1.resample('1s').asfreq().interpolate(method=interpolationType)
C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:7: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs2Interp = HRs2.resample('1s').asfreq().interpolate(method=interpolationType)


number of samples in each group 99791385 99791387
number of samples in common 99791382
total Squared Difference: 48054411326.0
481.5487105489731


C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:6: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs1Interp = HRs1.resample('1s').asfreq().interpolate(method=interpolationType)
C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:7: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs2Interp = HRs2.resample('1s').asfreq().interpolate(method=interpolationType)


number of samples in each group 16552143 16552146
number of samples in common 16552143
total Squared Difference: 64131147029.0
3874.4920841367793


C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:6: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs1Interp = HRs1.resample('1s').asfreq().interpolate(method=interpolationType)
C:\Users\abhik\AppData\Local\Temp\ipykernel_8028\4131151390.py:7: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  HRs2Interp = HRs2.resample('1s').asfreq().interpolate(method=interpolationType)


number of samples in each group 136078706 136078705
number of samples in common 136078705
total Squared Difference: 6977206943.0
51.273319679225345
